# Automatic Neuron Classification - Paper summary

We go through the basics of what the paper will be about:
* artificial dataset
* amanda's dataset
* chick data (tbd)
* allen brain institute data
## Table of contents
1. [Load and prepare artificial dataset](#introduction)
2. [Find optimum thresholds](#optimize)
3. [Adding new data to existing graph](#adddata)

#### todo:
* add raw data for ephys voltage traces
* make nicer initial 2d plots showing all different combinations of features -- b/c we only use constant thresholds, you could just do a line for each w/ scatterplot on each line and stack all lines -- one line per feature.
* Get better features in artificial data to see if adding more features can allow classification into more categories.
* how exactly to define "most stereotypical" neuron?

August 19th, 2021.

## Load and prepare artificial dataset <a name="introduction"></a>

First, we need to load the python packages that we'll be using.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import networkx as nx
import networkx.algorithms.community as nx_comm
from sklearn.model_selection import train_test_split

import util

Next, we need to load in the artificial dataset. This came from my own code based on the group's adult Zebra Finch HVC model; all parameters were varied by 20%.

In [ ]:
artificial_data = pd.read_csv("artificial_data.csv")
artificial_data.iloc[0:10]

The columns we care about for data analysis are the last 4, so let's isolate those. We also want to extract the labels. There are some datapoints that have `"None"` values for either average spike frequency or spike amplitude/width. This means that either there was only one spike, or no spike, respectively. We need to clean the data first. To do that, we eliminate all rows (neurons) that have a `"None"` value in the "first spike amplitude" -- this means they didn't spike.

(note: for this specific dataset all traces have at least one spike -- leaving this part in just in case for larger variation)

In [ ]:
artificial_data = artificial_data[~(artificial_data["first spike amplitude"] == "None")]
print(f"We now have {len(artificial_data.index)} data that do not contain `None` values in the first spike amplitude column")
artificial_data.iloc[15:25]

Right now, there is a `None` value for some neurons -- e.g. neuron 19  -- in `average spike frequency`. This means there was only a single spike for that trace. One way we can improve the data is to say that if there is a single spike, then set the frequency to zero.

We also -- from looking at this data before -- want to not include `first spike width` in our analysis. Including features that do not segment the data well will destroy the analysis.

In [ ]:
feature_names = ['resting membrane potential', 'average spike frequency']
X = artificial_data[feature_names]
y = artificial_data['label']

X['average spike frequency'].replace(to_replace="None", value=0.0, inplace=True)

# convert to float datatype to be able to plot later
X= X.astype(float, errors = 'raise')

For the purposes of this's data, we want to split it into a "train" and a "test" pool. This is so that we can find our "optimal" thresholds using the training data, then verify that our method works with the test data. The following function splits our data into 1/4 training, 3/4 testing. Typically we would want this ratio to be reversed, but because this is being presented "live", and the optimization algorithm takes a while, I'll keep it like this.

The random_state parameter lets us reproduce the "random" shuffling of data each time to make this notebook reproducable.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.75, random_state=42)

Because we have 4 features, we can choose 2 to plot these data as a scatterplot to get an intuitive idea of what the data looks like. Play around with the feature names index to get different plots and see how the labels depend on the classification.

In [ ]:
class_names = ['Int', 'RA', 'X']
for i in range(1,len(feature_names)):
    select_features = [feature_names[0], feature_names[i]]
    util.plot_2dfeature_space(X_train.loc[:,select_features], y_train, select_features, class_names)

We can see that the three classes are clearly segmented, but not by each feature. This is a reflection of the model data coming from three different parameter sets of the HVC neuron model. `first spike width`, for instance, does not seem to separate the data, at least between the `X` and `Int` classes.

We hope that our algorithm can classify activity from these three groups correctly. What we will end up doing is finding threshold values to generate a graph where the nodes are neurons and edges are number of similarities between neurons.

Consider the example of choosing a threshold of -70 for resting membrane potential, 60 for average spike frequency, and 95 for first spike amplitude.

In [ ]:
thresholds = [-70, 60]#, 95, 100]

In [ ]:
graphmat = util.compute_sym_graphmat(X_train, thresholds)

If we plot the adjacency matrix, we see that all values are either 0 through 4. This is because we only used 4 features, so the maximum amount of similarities was 4. This adjacency matrix doesn't mean much because the data has been shuffled, so the index values are not neatly segmented between the different classes of neurons.

In [ ]:
util.display_graphmat(graphmat)

We can also plot (the first two) features again with the threshold to see where they wind up.

In [ ]:
util.plot_2dfeature_space(X_train, y_train, feature_names, class_names, thresholds=thresholds)

## Find optimum thresholds <a name="optimize"></a>

We want to be able to find the correct thresholds systematically. Essentially, we will want to find the thresholds that minimize some "energy" function (defined later). 

There are several options to do this. First, we could just brute force the entire thing and compute the energy function over all our possible domain and select the minimum. This approach is not practical for problems with even moderate numbers of neurons or features. 

The second option would be to use a gradient descent-type method that follows the downward slope of the energy function to the minimum. Unfortunately for our problem, our domain has large regions of zero gradient -- i.e. changing the threshold values slightly will not affect the energy function. Basically, this will occur wherever changing the threshold values does not change the segmentation of the data.

Therefore, we go for a stochastic optimization method, which has the advantage of not being hindered by small regions of zero gradient. In fact, we never use the gradient in this method -- Differential Evolution [[wiki]](https://en.wikipedia.org/wiki/Differential_evolution) [[scipy implementation]](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.differential_evolution.html).

Regardless of our optimization approach, we'll have to define our bounds to help the algorithm search. This is just the respective min/max along each dimension.

In [ ]:
bounds = util.find_feature_bounds(X_train)
for (i, val) in enumerate(feature_names):
    print(f"The bounds of {val} are ({bounds[i][0]},{bounds[i][1]})")

The interior of these bounds is the space of all possible thresholds. We want to minimize some energy function over this space. We now need to choose our 'energy' function that we will be minimizing. We will choose a function that computes the modularity of a graph partitioned by the [Girvin-Newman](https://en.wikipedia.org/wiki/Girvan%E2%80%93Newman_algorithm) algorithm.

All of this functionality is defined in our custom module `util.py`. Right now we'll just call it to see some results. This next line might take some time.

In [ ]:
opt_thresh, opt_val = util.my_diff_evolution(X_train, bounds)

`opt_thresh` is the set of thresholds that give the minimum each dimension. `opt_val` is the actual value of this minimum in case we want it to check something (we don't need it, or use it). We can use `opt_thresh` to generate the network and examine how it performed.

In [ ]:
util.plot_2dfeature_space(X_train, y_train, feature_names, class_names, thresholds=opt_thresh)

In [ ]:
Astar = util.compute_sym_graphmat(X_train, opt_thresh)

In [ ]:
G = nx.Graph(Astar)
nx.draw(G, with_labels=True)

In [ ]:
comm = next(nx_comm.girvan_newman(G))
mod = nx_comm.quality.modularity(G, comm)
print(f"The graph has {len(comm)} communities")
print(f"Modularity for this partision is {mod}")
color_map = util.get_color_map(G, comm)
nx.draw(G, node_color=color_map, with_labels=True)

In [ ]:
pred = util.get_pred(G, comm)
print("Pred\tLabel")
for i, val in enumerate(pred):
    print(f"{val}\t{y_train.iloc[i]}")

In [ ]:
fig, ax = plt.subplots()
colors = (np.unique(y_train, return_inverse=True)[1])
scatter = plt.scatter(X_train[feature_names[0]], X_train[feature_names[1]], c=color_map,
                     cmap = plt.get_cmap('jet'), alpha=0.3, s=215)

## Adding new data to existing graph <a name="adddata"></a>
So we are confident now that our approach works reasonably well, except we need to add more features. For now, let's just see if this chosen threshold will correctly categorize the training data. Note that we don't have to re-run the optimization process -- we assume the thresholds found in the "training" set is good for the "test" set.

In the real world, the training set will be some existing (unlabeled) dataset, and the test set will simply be the addition of new data to an existing graph.

Let's first make sure we know what the test data looks like.

In [ ]:
util.plot_2dfeature_space(X_test, y_test, feature_names, class_names, thresholds=opt_thresh)

So, by coincidence, it looks like the RA neurons of the test data have a higher spike frequency than those in the training set. Nevertheless, we hope that the addition of the training data to the test data can be groups as before. We will use the pandas `concat` function to combine X_train and X_test to mimick the effect of the original `X` being unavailable.

In [ ]:
X_all = pd.concat([X_train, X_test], axis=0)
y_all = pd.concat([y_train, y_test], axis=0)
Astar = util.compute_sym_graphmat(X_all, opt_thresh)
G = nx.Graph(Astar)
comm = next(nx_comm.girvan_newman(G))
print(f'This graph has {len(comm)} communities.')
color_map = util.get_color_map(G, comm)
nx.draw(G, node_color=color_map, with_labels=True)

In [ ]:
pred = util.get_pred(G, comm)
print("Pred\tLabel")
for i, val in enumerate(pred):
    print(f"{val}\t{y_all.iloc[i]}")

So again all area X neurons were assigned to the `0` community, and RA and Interneurons were assigned to the `1` community.